# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame 
city_data_df = pd.read_csv(r"C:\Users\greci\Desktop\BOOTCAMP\GITLAB WORKING FOLDER\06 - Python APIs Unit\Module 06 Challenge - Due 11-26-2024\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.57,75,97,16.85,TF,1741461934
1,1,bredasdorp,-34.5322,20.0403,21.44,87,0,2.78,ZA,1741461934
2,2,ushuaia,-54.8000,-68.3000,8.81,66,75,10.29,AR,1741461437
3,3,xiaolingwei,32.0333,118.8500,6.34,76,89,3.84,CN,1741461934
4,4,sandnessjoen,66.0217,12.6316,3.35,64,100,6.93,NO,1741461934


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:

# YOUR CODE HERE
city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True, 
    tiles="OSM", 
    size="Humidity",  
    color="Humidity",  
    colormap="viridis",  
    alpha=0.7,
    hover_cols=["City", "Country", "Humidity"]  # Show details on hover
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,bredasdorp,-34.5322,20.0403,21.44,87,0,2.78,ZA,1741461934
20,20,port elizabeth,-33.9180,25.5701,25.73,86,0,1.03,ZA,1741461937
52,52,esikhaleni,-28.8710,31.8996,23.13,87,0,4.16,ZA,1741461731
65,65,sittwe,20.1500,92.9000,22.81,61,0,1.38,MM,1741461945
80,80,yeppoon,-23.1333,150.7333,24.51,92,0,1.80,AU,1741461948


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,bredasdorp,ZA,-34.5322,20.0403,87,
20,port elizabeth,ZA,-33.9180,25.5701,86,
52,esikhaleni,ZA,-28.8710,31.8996,87,
65,sittwe,MM,20.1500,92.9000,61,
80,yeppoon,AU,-23.1333,150.7333,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
import requests

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": "52f091b56eb245a9b9ce3afe7c546c24"
}

# Print a message to follow up the hotel search
print("Hotel search in progress")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Hotel search in progress
bredasdorp - nearest hotel: Victoria Hotel
port elizabeth - nearest hotel: Waterford Hotel
esikhaleni - nearest hotel: No hotel found
sittwe - nearest hotel: Yuzana Aung Motel 1
yeppoon - nearest hotel: Blossom's on Seaspray
cabo san lucas - nearest hotel: Comfort Rooms
sukuta - nearest hotel: London Inn
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
holualoa - nearest hotel: Kona Hotel
ubari - nearest hotel: فندق اوباري
reggane - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
port lincoln - nearest hotel: Boston Hotel
veraval - nearest hotel: Shyam
razole - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
pisco - nearest hotel: La Portada
kailua-kona - nearest hotel: PACIFIC 19 Kona
pipar - nearest hotel: Ganesh Hotel
ghadamis - nearest hotel: فندق عين الفارس
ampanihy - nearest hotel: Hotel Restaurant Maison-Blance
metlili chaamba - nearest hotel: No hotel found
mandal - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
1,bredasdorp,ZA,-34.5322,20.0403,87,Victoria Hotel
20,port elizabeth,ZA,-33.9180,25.5701,86,Waterford Hotel
52,esikhaleni,ZA,-28.8710,31.8996,87,No hotel found
65,sittwe,MM,20.1500,92.9000,61,Yuzana Aung Motel 1
80,yeppoon,AU,-23.1333,150.7333,92,Blossom's on Seaspray
100,cabo san lucas,MX,22.8909,-109.9124,45,Comfort Rooms
103,sukuta,GM,13.4103,-16.7082,56,London Inn
133,morehead city,US,34.7229,-76.7260,69,"Bask Hotel Big Rock Landing, Trademark Collect..."
182,holualoa,US,19.6228,-155.9522,68,Kona Hotel
216,ubari,LY,26.5921,12.7805,37,فندق اوباري


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat", 
    geo=True, 
    tiles="OSM", 
    size=100, 
    color="Humidity", 
    colormap="plasma", 
    alpha=0.8, 
    hover_cols=["Hotel Name", "City", "Country"]  # Added hotel name & country to hover message
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,City,Country)